# DFX + DMA + self trigger

## import section

In [ ]:
# import the library
from pynq import Overlay     # import the overlay
from pynq import allocate    # import for CMA (contingeous memory allocation)
from pynq import DefaultIP   # import the ip connector library for extension
import numpy as np
import os
import subprocess
import re

PRJ_DIR    = '/home/xilinx/jupyter_notebooks/dfx4ml/'
PRJ_HW_DIR = '/home/xilinx/jupyter_notebooks/dfx4ml/hw/'

DFX_CONFIG_FILE = 'dfxCtrlMeta.txt'

FULL_BS_NAME    = 'system.bin'
PAR_BS_NAME_0   = 'add.bin'
PAR_BS_NAME_1   = 'sub.bin'
PAR_BS_NAME_ML1 = 'ml1.bin'
PAR_BS_NAME_ML2 = 'ml2.bin'
PAR_BS_NAME_ML3 = 'ml3.bin'

AMT_QUERY       = 5
INPUT_SHAPE     = (AMT_QUERY, 6, 8 )
ML2IN_SHAPE     = (AMT_QUERY, 6, 16)
ML3IN_SHAPE     = (AMT_QUERY, 6, 8 )
DES_SHAPE       = (AMT_QUERY, 5)

## ICAP config

In [2]:
def subProcessPrint(prefix, subProcessRes):
    
    print(f"{prefix} STDOUT:", subProcessRes.stdout)
    print(f"{prefix} ERROR :", subProcessRes.stderr)
    print("--------------------------------")
    

def changePLconfigMode(mode, isRoot): ### mode should be "pcap", "icap"
    indicator = "1"
    
    if mode == "icap":
        indicator = "0"
    elif mode == "pcap":
        pass
    else:
        raise Exception("change PL config Mode has mode error")
    
    
    changeCmd_inputVal = f"0xFFCA3008 0xFFFFFFFF 0x{indicator}"
    triggerCmd_inputVal = "0xFFCA3008"
    config_file = "/sys/firmware/zynqmp/config_reg"
    password = "xilinx"  # default sudo password for PYNQ-ZU

    # Compose the command string
    cmd_change = f"sudo -S tee {config_file}"
    cmd_trigger = f"sudo -S tee {config_file}"
    cmd_read    = f"cat {config_file}"
    
    passwordCmd = (password + '\n') if not isRoot else ""

    # Run the change command
    result = subprocess.run(
        cmd_change,
        input=passwordCmd + changeCmd_inputVal + '\n',
        shell=True,
        capture_output=True,
        text=True
    )
    subProcessPrint("CHANGE CMD", result)

    # Run the trigger command
    result = subprocess.run(
        cmd_trigger,
        input=passwordCmd + triggerCmd_inputVal + '\n',
        shell=True,
        capture_output=True,
        text=True
    )

    subProcessPrint("TRIGGER CMD", result)

    
    result = subprocess.run(
        cmd_read,
        shell=True,
        capture_output=True,
        text=True
    )

    subProcessPrint("READ CMD", result)


changePLconfigMode("pcap", True)  # change the PL config mode to pcap

CHANGE CMD STDOUT: 0xFFCA3008 0xFFFFFFFF 0x1

CHANGE CMD ERROR : 
--------------------------------
TRIGGER CMD STDOUT: 0xFFCA3008

TRIGGER CMD ERROR : 
--------------------------------
READ CMD STDOUT: 0x1

READ CMD ERROR : 
--------------------------------


## SAFE IO

In [ ]:
class SafeIoIp (DefaultIP):

    def __init__(self, description, icheckOnRead = True, icheckOnWrite = True):
        super().__init__(description=description)
        self.checkOnRead  = icheckOnRead
        self.checkOnWrite = icheckOnWrite

    def confirmation(self, shouldCheck):
        if shouldCheck:
            while True:
                answer = input("Please enter yes or no: ").strip().lower()
                if answer in ['yes', 'y']:
                    print("You answered YES")
                    break
                elif answer in ['no', 'n']:
                    raise Exception("user dicides to reject operation")
                else:
                    print("Invalid input. Try again.")

    def setCheckOnRead(self, iCheckOnRead):
        self.checkOnRead = iCheckOnRead

    def setCheckOnWrite(self, iCheckOnWrite):
        self.checkOnWrite = iCheckOnWrite

    def safeRead(self, addr):
        self.confirmation(self.checkOnRead)
        return self.read(addr)

    def safeWrite(self, addr, data):
        self.confirmation(self.checkOnWrite)
        self.write(addr, data)

## Magic Sequence Driver

In [ ]:
class MagicSeqdDriver(SafeIoIp):

    def __init__(self, description):
        super().__init__(description=description)

        ### Bit Layout start bit
        self.BL_COL_ST  =  2 
        self.BL_ROW_ST  =  6 
        self.BL_BNK_ST  = 14 
        ### Bit Layout size
        self.BIT_COL_SZ = 4
        self.BIT_ROW_SZ = 8
        self.BIT_BNK_SZ = 2

        self.REG_CTRL     = (0,0,0)
        self.REG_ST       = (0,1,0)
        self.REG_MAINCNT  = (0,2,0)
        self.REG_ENDCNT   = (0,3,0)
        self.REG_DMA_ADDR = (0,4,0)
        self.REG_DFX_ADDR = (0,5,0)

        #### the row must be change to match the slot
        self.SLOT_SRC_ADDR = (1,0,0)
        self.SLOT_SRC_SIZE = (1,0,1)
        self.SLOT_DES_ADDR = (1,0,2)
        self.SLOT_DES_SIZE = (1,0,3)
        self.SLOT_STATUS   = (1,0,4)
        self.SLOT_PROF     = (1,0,5)


        self.LIM_AMT_SLOT = 4 ### limit amount slot
        

    bindto = ['user.org:user:MagicSeqTop:1.0']

    def genAddr(self, bankId, rowIdx, colIdx):
        return (bankId << self.BL_BNK_ST) | (rowIdx << self.BL_ROW_ST) | (colIdx << self.BL_COL_ST)
    
    def genAddrForSlot(self, slotT, slotIdx):
        return self.genAddr(slotT[0], slotIdx, slotT[2])

    ###############################################
    ####### getter ################################
    ###############################################

    def getStatus(self):
        return self.safeRead(self.genAddr(*self.REG_ST))
    def getMainCnt(self):
        return self.safeRead(self.genAddr(*self.REG_MAINCNT))
    def getEndCnt(self):
        return self.safeRead(self.genAddr(*self.REG_ENDCNT))
    def getDmaAddr(self):
        return self.safeRead(self.genAddr(*self.REG_DMA_ADDR))
    def getDfxAddr(self):
        return self.safeRead(self.genAddr(*self.REG_DFX_ADDR))
    
    def getSlot(self, slotIdx):

        addr_srcAddr  = self.genAddrForSlot(self.SLOT_SRC_ADDR, slotIdx)             
        addr_srcSz    = self.genAddrForSlot(self.SLOT_SRC_SIZE, slotIdx)           
        addr_desAddr  = self.genAddrForSlot(self.SLOT_DES_ADDR, slotIdx)             
        addr_desSz    = self.genAddrForSlot(self.SLOT_DES_SIZE, slotIdx)           
        addr_status   = self.genAddrForSlot(self.SLOT_STATUS  , slotIdx)            
        addr_prof     = self.genAddrForSlot(self.SLOT_PROF    , slotIdx)

        data_srcAddr  = self.safeRead(addr_srcAddr)
        data_srcSz    = self.safeRead(addr_srcSz)
        data_desAddr  = self.safeRead(addr_desAddr)
        data_desSz    = self.safeRead(addr_desSz)
        data_status   = self.safeRead(addr_status)
        data_prof     = self.safeRead(addr_prof)

        return data_srcAddr, data_srcSz, data_desAddr, data_desSz, data_status, data_prof
        

    ###############################################
    ####### setter ################################
    ###############################################

    def setControl(self, value): #### status registesr will be neglect
        return self.safeWrite(self.genAddr(*self.REG_CTRL), value)
    # def setMainCnt(self, value):
    #     return self.safeWrite(self.genAddr(*self.REG_MAINCNT), value)
    def setEndCnt(self, value):
        return self.safeWrite(self.genAddr(*self.REG_ENDCNT), value)
    def setDmaAddr(self, value):
        return self.safeWrite(self.genAddr(*self.REG_DMA_ADDR), value)
    def setDfxAddr(self, value):
        return self.safeWrite(self.genAddr(*self.REG_DFX_ADDR), value)
    
    def setSlot(self, slotT, slotIdx, value):
        addr  = self.genAddrForSlot(slotT, slotIdx) 
        self.safeWrite(addr, value)

    def setWholeSlot(self, slotIdx, dataList):

        addr_srcAddr  = self.genAddrForSlot(self.SLOT_SRC_ADDR, slotIdx)             
        addr_srcSz    = self.genAddrForSlot(self.SLOT_SRC_SIZE, slotIdx)           
        addr_desAddr  = self.genAddrForSlot(self.SLOT_DES_ADDR, slotIdx)             
        addr_desSz    = self.genAddrForSlot(self.SLOT_DES_SIZE, slotIdx)           
        addr_status   = self.genAddrForSlot(self.SLOT_STATUS  , slotIdx)            
        addr_prof     = self.genAddrForSlot(self.SLOT_PROF    , slotIdx)

        self.safeWrite(addr_srcAddr, dataList[0])
        self.safeWrite(addr_srcSz  , dataList[1])
        self.safeWrite(addr_desAddr, dataList[2])
        self.safeWrite(addr_desSz  , dataList[3])
        self.safeWrite(addr_status , dataList[4])
        self.safeWrite(addr_prof   , dataList[5])

    ###############################################
    ####### command################################
    ###############################################

    def clearEngine(self):
        print("[cmd] clear the engine")
        self.setControl(0)
        print("[cmd] clear the engine successfully")


    def shutdownEngine(self):
        print("[cmd] shutdown the engine")
        self.setControl(1)
        print("[cmd] shutdown successfully")

    def startEngine(self):
        print("[cmd] start the engine")
        self.setControl(2)
        print("[cmd] start the successfully")

    ###############################################
    ####### debugger ##############################
    ###############################################

    def status2Str(self, statusIdx):
        mapper = ["SHUTDOWN","REPROG","W4SLAVERESET","W4SLAVEOP","INITIALIZING","TRIGGERING","WAIT4FIN","PAUSEONERROR"]

        if statusIdx not in range(0, len(mapper)):
            return "STATUS ERROR"
        return mapper[statusIdx]
        
    def printMainStatus(self):


        print("----- MAIN STATUS ------------------")
        status  = self.getStatus()
        print("--------> STATUS = ", self.status2Str(status))
        mainCnt = self.getMainCnt()
        print("--------> MAINCNT = ", mainCnt)
        endCnt  = self.getEndCnt()
        print("--------> ENDCNT  = ", endCnt)
        dmaAddr = self.getDmaAddr()
        print("--------> DMAADDR  = ", hex(dmaAddr))
        dfxAddr = self.getDfxAddr()
        print("--------> DFXADDR  = ", hex(dfxAddr))


    def printSlotData(self):

        print("----- SLOT DATA ------------------")

        if self.getStatus() != 0:
            print("---------- cannot print slot data due to the system is not in shutdown state")
            return

        for slotIdx in range (self.LIM_AMT_SLOT):
            s_addr, s_size, d_addr, d_size, status, prof = self.getSlot(slotIdx)

            print(f"------> slot {slotIdx} :")
            print(f"        srcAddr   : {hex(s_addr)},  srcSize   : {hex(s_size)}")
            print(f"        desAddr   : {hex(d_addr)},  desSize   : {hex(d_size)}")
            print(f"        status    : {hex(status)}")
            print(f"        profileCnt: {hex(prof)}")

    def printDebug(self):
        self.printMainStatus()
        self.printSlotData()
        print("-------------------------------")

## DFX controller Driver

In [ ]:
class MyDfxCtrl(SafeIoIp):
    def __init__(self, description):
        self.isMetaConfigured = False
        super().__init__(description=description)
        self.storage = None
        # BLS bit layout size
        self.BLS_DATA   = 2 # register contain 4 byte (2 bit addressing)
        self.BLS_REGID  = 4 # register Id
        self.BLS_BANKID = 2 # bank id
        #### TODO this may change 
        # GENERAL BANK
        self.BANK_GENREG      = 0     
        self.GENREG_STATUS    = 0
        self.GENREG_CTRL      = 0
        self.GENREG_SWTRIGGER = 1
        # TRIGGER RM MAPPING
        self.BANK_RMM         = 1
        self.BANK_RMM_LIMIT   = 2 #### it is not used now!
        # RM INFO
        self.BANK_RMINFO      = 2
        self.BANK_RMINFO_LIMIT= 2 #### it is not used now!
        # BITSTREAM INFO
        self.BANK_BSINFO      = 3
        self.BANK_BSINFO_LIMIT= 2 #### it is not used now!

        ##### retrive the actual metadata

        
        #print(description) #
        
    bindto = ['xilinx.com:ip:dfx_controller:1.0']

    def config(self, metaPath):

        regBankIdx, regColIdx = self.retrieveConfig(metaPath)
        self.BLS_REGID  = regColIdx [1] - regColIdx [0] + 1
        self.BLS_BANKID = regBankIdx[1] - regBankIdx[0] + 1
        

    #######################################
    ###### config check ###################
    #######################################
    def retrieveConfig(self, path):
        regBankIdx = None
        regColIdx  = None

        with open(path, 'r') as file:
            for line in file:
                if "Selects the Register Bank" in line:
                    regBankIdx = self.extractBitRange(line)
                    print(f"regbank detect index {regBankIdx}")
                    
                if "Selects the Register within the bank" in line:
                    regColIdx = self.extractBitRange(line)
                    print(f"regbank detect index {regColIdx}")

        return regBankIdx, regColIdx



    def extractBitRange(self, line):

        match = re.search(r'\[\s*(\d+)\s*:\s*(\d+)\s*\]', line)

        if match:
            high = int(match.group(1))
            low  = int(match.group(2))
            return low, high
        else:
            return None


    #######################################
    ###### address generator ##############
    #######################################
    def getAddress(self, bankId, regId): ### todo make it compatible for more than 1 slot
        if not self.isMetaConfigured:
            raise Exception("this module has not initialized yet, any attempt to interact with the IP will be abort")
        return (bankId << (self.BLS_DATA + self.BLS_REGID)) + (regId << (self.BLS_DATA))
    #######################################
    ###### general command ################
    #######################################
    def shutdownEngine(self):
        print("shutdown dfx Controller")
        self.setCtrl(0)
    
    def restartNoStatus(self):
        print("restart the dfx Controller with no status")
        self.setCtrl(1)
        
    def restartWithStatus(self):
        print("restart the dfx Controller with status")
        
    def trigger(self, triggerId):
        print("trig the rmId ", triggerId)
        self.setCtrlTrigger(triggerId)

    #######################################
    ###### getter setter command ##########
    #######################################
        
    ###### general register bank0
    #
    # |statusRegister, controlRegister|
    # | trigger register              |
    #
    def getStatus(self):
        regAddr = self.getAddress(self.BANK_GENREG, self.GENREG_STATUS)
        print("[get status register] @", hex(regAddr))
        return self.safeRead(regAddr)
    
    def getCtrl(self):
        regAddr = self.getAddress(self.BANK_GENREG, self.GENREG_CTRL)
        print("[get ctrl register] @", hex(regAddr))
        return self.safeRead(regAddr)
    
    def setCtrl(self, command):
        regAddr = self.getAddress(self.BANK_GENREG, self.GENREG_CTRL)
        print("[set ctrl register] @", hex(regAddr), " with command ", hex(command))
        self.safeWrite(regAddr, command)
    
    def getCtrlTrigger(self):
        regAddr = self.getAddress(self.BANK_GENREG, self.GENREG_SWTRIGGER)
        print("[get Ctrl Trigger] @", hex(regAddr))
        return self.safeRead(regAddr)
    
    def setCtrlTrigger(self, triggerId):
        regAddr = self.getAddress(self.BANK_GENREG, self.GENREG_SWTRIGGER)
        print("[set Ctrl Trigger] @", hex(regAddr))
        self.safeWrite(regAddr, triggerId)
        
    ####### Reconfig Module Map bank1
    # | RM0 |
    # | RM1 |
    # | RM2 |
    # .
    def getRMM(self, triggerId):
        regAddr = self.getAddress(self.BANK_RMM, triggerId)
        print("[get RM MAP] @", hex(regAddr))
        return self.safeRead(regAddr)
    
    def setRMM(self, triggerId, infoId):
        regAddr = self.getAddress(self.BANK_RMM, triggerId)
        print("[set RM MAP] @", hex(regAddr), " info ", hex(infoId))
        self.safeWrite(regAddr, infoId)
        
    ####### Reconfig Module Map bank2
    # | BS_ID0 | CT_ID0 |
    # | BS_ID1 | CT_ID1 |
    # | BS_ID2 | CT_ID2 |
    # .
    def getRMInfo(self, infoId):
        bsIdxAddr = self.getAddress(self.BANK_RMINFO, infoId * 2)
        ### *2 because each row has two element
        ctrlAddr  = self.getAddress(self.BANK_RMINFO, (infoId * 2) + 1)
        print("[get RM INFO] bsIdxAddr@", hex(bsIdxAddr), " ctrlAddr@", hex(ctrlAddr))
        return (self.safeRead(bsIdxAddr), self.safeRead(ctrlAddr))
    
    def setRMInfo(self, infoId, bsIdx, ctrlCmd):
        ### *2 because each row has two element
        bsIdxAddr = self.getAddress(self.BANK_RMINFO, infoId * 2)
        ctrlAddr  = self.getAddress(self.BANK_RMINFO, (infoId * 2) + 1)
        print("[get RM INFO] bsIdxAddr@", hex(bsIdxAddr), " ctrlAddr@", hex(ctrlAddr))
        self.safeWrite(bsIdxAddr, bsIdx)
        self.safeWrite(ctrlAddr, ctrlCmd)
    ####### BIN stream bank3
    # | BIN_ADDR0 | SIZE0 |
    # | BIN_ADDR1 | SIZE1 |
    # | BIN_ADDR2 | SIZE2 |
    # .
    def getBSInfo(self, bsId):
        ### *3 because each row has three element
        streamIdentAddr  = self.getAddress(self.BANK_BSINFO, (bsId * 4)    )
        streamAddr       = self.getAddress(self.BANK_BSINFO, (bsId * 4) + 1)
        sizeAddr         = self.getAddress(self.BANK_BSINFO, (bsId * 4) + 2)
        print("[get BS INFO] streamAddr@", hex(streamAddr), " sizeAddr@", hex(sizeAddr))
        return (self.safeRead(streamIdentAddr), self.safeRead(streamAddr), self.safeRead(sizeAddr))
    
    def setBSInfo(self, bsId, phyStreamAddr, streamSize):
    
        streamIdentAddr  = self.getAddress(self.BANK_BSINFO, (bsId * 4)    )
        streamAddr       = self.getAddress(self.BANK_BSINFO, (bsId * 4) + 1)
        sizeAddr         = self.getAddress(self.BANK_BSINFO, (bsId * 4) + 2)
        print("[get BS INFO] streamAddr@", hex(streamAddr), " sizeAddr@", hex(sizeAddr))
        self.safeWrite(streamIdentAddr, 1)
        self.safeWrite(streamAddr     , phyStreamAddr)
        self.safeWrite(sizeAddr       , streamSize)
        
    ######## AUTO META DATA RECONFIGURE
    def setSimpleMetaData(self, idx, streamPhyAddr, streamPhySize):
        
        print("setting RM Mapping to ", idx)
        self.setRMM(idx, idx)
        print("setting RM INFO to ", idx)
        ctrlValue = 0B0_10_0_00
        print("control value for active low reset is ", hex(ctrlValue))
        self.setRMInfo(idx, idx, ctrlValue)
        
        print("setting BS INFO to ", idx, " with streamAddress: ", streamPhyAddr, " with size: ", streamPhySize)
        self.setBSInfo(idx, streamPhyAddr, streamPhySize)
    
    ###########################################
    ######## DEBUGGER #########################
    ###########################################

    def printStatus(self):
        
        status = self.getStatus()
        
        print(">>status of the system vs0")
        print("-------> Is device shutdown: ", (status >> 7) & 0x1)
        print("-------> current error code: ", hex((status >> 3) & 0xF))
        print("-------> active RM_ID      : ", hex((status >> 8) & 0xFFFF))
        print("-------> state      : ", hex(status & 0x7))
        
    def printSimpleMetaData(self, idx):
        print("get metadata info for row", idx)
        print("RM MAPPER: ", self.getRMM   (idx))
        print("RM INFO  : ", self.getRMInfo(idx))
        print("BS INFO  : ", self.getBSInfo(idx))
        
        
        
    ###############################################
    ####### ALLOCATE BIN STREAM ON MAIN MEMORY ####
    ###############################################
    
    def allocateBitStreamCMA(self, path):
              
        print(">>allocateBitStream")
        
        print("opening file ", path)
        with open(path, 'rb') as f:
            data = f.read()
        file_size = len(data)
        print("copying the data")
        data_u32  = np.frombuffer(data, dtype='<u4')  # Big-endian uint32
        buffer    = allocate(shape=(len(data_u32),), dtype='>u4')
        buffer[:] = data_u32
        print("copy complete")
        print("file size ", file_size)
        print("---------------------------------")
        return buffer, buffer.physical_address, file_size

## memory allocation

In [ ]:
def allocDataUint(allocShape = (16, ), allocType = np.float32, inputX = None):
    buf0 = allocate(shape=allocShape, dtype=allocType)
    #### copy the data
    if inputX is not None:
        print("start copy from input to allocate buffer")
        if (allocShape != inputX.shape) or (allocType != inputX.dtype):
            raise Exception("the specified shape and inputX shape is mismatch")
        np.copyto(buf0, inputX)
        print("copy finish")

    return buf0, buf0.physical_address, buf0.nbytes

## main procedure

In [ ]:
AMT_SLOT = 3

In [7]:
#### change configmode
changePLconfigMode("pcap", True)

CHANGE CMD STDOUT: 0xFFCA3008 0xFFFFFFFF 0x1

CHANGE CMD ERROR : 
--------------------------------
TRIGGER CMD STDOUT: 0xFFCA3008

TRIGGER CMD ERROR : 
--------------------------------
READ CMD STDOUT: 0x1

READ CMD ERROR : 
--------------------------------


In [8]:
#### load overlay
overlay  = Overlay(PRJ_HW_DIR + FULL_BS_NAME)

In [ ]:
#### get the device
dmaIp      = overlay.axi_dma_0
dfxCtrlIp  = overlay.dfx_controller_0
magicSeqIp = overlay.MagicSeqTop_0

In [10]:
### change reconfigure mode
changePLconfigMode("icap", True)

CHANGE CMD STDOUT: 0xFFCA3008 0xFFFFFFFF 0x0

CHANGE CMD ERROR : 
--------------------------------
TRIGGER CMD STDOUT: 0xFFCA3008

TRIGGER CMD ERROR : 
--------------------------------
READ CMD STDOUT: 0x0

READ CMD ERROR : 
--------------------------------


In [11]:
dfxCtrlIp.printStatus()

[get status register] @ 0x0
>>status of the system vs0
-------> Is device shutdown:  1
-------> current error code:  0x0
-------> active RM_ID      :  0x0
-------> state      :  0x1


In [12]:
#### shutdown all system

magicSeqIp.shutdownEngine()
dfxCtrlIp.shutdownEngine()

[cmd] shutdown the engine
[cmd] shutdown successfully
shutdown dfx Controller
[set ctrl register] @ 0x0  with command  0x0


In [13]:
##### set the sequencer
dmaPhyAddr     =  overlay.ip_dict['axi_dma_0']['phys_addr']
dfxCtrlPhyAddr =  overlay.ip_dict['dfx_controller_0']['phys_addr']

print("dma physical address: ", hex(dmaPhyAddr))
print("dfx  Ctrl physical address: ", hex(dfxCtrlPhyAddr))

dma physical address:  0xa0000000
dfx  Ctrl physical address:  0xa0010000


In [ ]:
##### initialize magic seq
print("------ before init magic seq------")
print(magicSeqIp.printDebug())

print("------ init magic sequence METADATA bank 0 -------------------------")
magicSeqIp.setEndCnt(AMT_SLOT-1) ### use the last index
magicSeqIp.setDmaAddr(dmaPhyAddr)
magicSeqIp.setDfxAddr(dfxCtrlPhyAddr)
inputX = np.random.rand(*INPUT_SHAPE).astype(np.float32)
print("-------------init all data buffer -------------")
buf_input   , buf_input_phya   , buf_input_sz    = allocDataUint(allocShape= INPUT_SHAPE, allocType= np.float32, inputX = inputX)
buf_ml2_in  , buf_ml2_in_phya  , buf_ml2_in_sz   = allocDataUint(allocShape= ML2IN_SHAPE, allocType= np.float32)
buf_ml3_in  , buf_ml3_in_phya  , buf_ml3_in_sz   = allocDataUint(allocShape= ML3IN_SHAPE  , allocType= np.float32)
buf_out     , buf_out_phy      , buf_out_sz      = allocDataUint(allocShape= DES_SHAPE  , allocType= np.float32)

print("------------- init all bank 1 ------------------")
magicSeqIp.setWholeSlot(0, [buf_input_phya , buf_input_sz , buf_ml2_in_phya, buf_ml2_in_sz, 0, 0])
magicSeqIp.setWholeSlot(1, [buf_ml2_in_phya, buf_ml2_in_sz, buf_ml3_in_phya, buf_ml3_in_sz, 0, 0])
magicSeqIp.setWholeSlot(2, [buf_ml3_in_phya, buf_ml3_in_sz, buf_out_phy    , buf_out_sz   , 0, 0])


print("------------- after init magic seq------")
print(magicSeqIp.printDebug())

------ before init magic seq------
----- MAIN STATUS ------------------
--------> STATUS =  SHUTDOWN
--------> MAINCNT =  0
--------> ENDCNT  =  0
--------> DMAADDR  =  0x0
--------> DFXADDR  =  0x0
----- SLOT DATA ------------------
------> slot 0 :
        srcAddr   : 0x0,  srcSize   : 0x0
        desAddr   : 0x0,  desSize   : 0x0
        status    : 0x0
        profileCnt: 0x0
------> slot 1 :
        srcAddr   : 0x0,  srcSize   : 0x0
        desAddr   : 0x0,  desSize   : 0x0
        status    : 0x0
        profileCnt: 0x0
------> slot 2 :
        srcAddr   : 0x0,  srcSize   : 0x0
        desAddr   : 0x0,  desSize   : 0x0
        status    : 0x0
        profileCnt: 0x0
------> slot 3 :
        srcAddr   : 0x0,  srcSize   : 0x0
        desAddr   : 0x0,  desSize   : 0x0
        status    : 0x0
        profileCnt: 0x0
-------------------------------
None
-------------allocFirstBuffer-------------
-------------allocSecondBuffer-------------
------ after init magic seq------
----- MAIN S

In [ ]:
##### initialize dfx controller
print("------ allocate bit steram CMA for each trigger ------")

######## set trigger 0
ml1_ip_buf, ml1_addr, ml1_size = \
    dfxCtrlIp.allocateBitStreamCMA(PRJ_HW_DIR + PAR_BS_NAME_ML1)
######## set trigger 1
ml2_ip_buf, ml2_addr, ml2_size = \
    dfxCtrlIp.allocateBitStreamCMA(PRJ_HW_DIR + PAR_BS_NAME_ML2)
######## set trigger 2
ml3_ip_buf, ml3_addr, ml3_size = \
    dfxCtrlIp.allocateBitStreamCMA(PRJ_HW_DIR + PAR_BS_NAME_ML3)

------ before init dfx ctrl ip------
>>allocateBitStream
opening file  /home/xilinx/jupyter_notebooks/magicFull0/hw/add.bin
copying the data
copy complete
file size  1000720
---------------------------------
------ after  init dfx ctrl ip------
>>allocateBitStream
opening file  /home/xilinx/jupyter_notebooks/magicFull0/hw/sub.bin
copying the data
copy complete
file size  1000720
---------------------------------


In [ ]:
##### initialize dfx controller2
dfxCtrlIp.setSimpleMetaData(0, ml1_addr, ml1_size)
dfxCtrlIp.setSimpleMetaData(1, ml2_addr, ml2_size)
dfxCtrlIp.setSimpleMetaData(2, ml3_addr, ml3_size)

setting RM Mapping to  0
[set RM MAP] @ 0x20  info  0x0
setting RM INFO to  0
control value for active low reset is  0x10
[get RM INFO] bsIdxAddr@ 0x40  ctrlAddr@ 0x44
setting BS INFO to  0  with streamAddress:  2017460224  with size:  1000720
[get BS INFO] streamAddr@ 0x64  sizeAddr@ 0x68
setting RM Mapping to  1
[set RM MAP] @ 0x24  info  0x1
setting RM INFO to  1
control value for active low reset is  0x10
[get RM INFO] bsIdxAddr@ 0x48  ctrlAddr@ 0x4c
setting BS INFO to  1  with streamAddress:  2018508800  with size:  1000720
[get BS INFO] streamAddr@ 0x74  sizeAddr@ 0x78


In [ ]:
##### check dfx controller3
dfxCtrlIp.printStatus()
dfxCtrlIp.printSimpleMetaData(0)
dfxCtrlIp.printSimpleMetaData(1)
dfxCtrlIp.printSimpleMetaData(2)

[get status register] @ 0x0
>>status of the system vs0
-------> Is device shutdown:  1
-------> current error code:  0x0
-------> active RM_ID      :  0x0
-------> state      :  0x1
get metadata info for row 0
[get RM MAP] @ 0x20
RM MAPPER:  0
[get RM INFO] bsIdxAddr@ 0x40  ctrlAddr@ 0x44
RM INFO  :  (0, 16)
[get BS INFO] streamAddr@ 0x64  sizeAddr@ 0x68
BS INFO  :  (0, 2017460224, 1000720)
get metadata info for row 1
[get RM MAP] @ 0x24
RM MAPPER:  1
[get RM INFO] bsIdxAddr@ 0x48  ctrlAddr@ 0x4c
RM INFO  :  (1, 16)
[get BS INFO] streamAddr@ 0x74  sizeAddr@ 0x78
BS INFO  :  (0, 2018508800, 1000720)


In [ ]:
dfxCtrlIp.trigger(0)
dfxCtrlIp.restartNoStatus()

trig the rmId  0
[set Ctrl Trigger] @ 0x4
restart the dfx Controller with no status
[set ctrl register] @ 0x0  with command  0x1


In [19]:
##### start dfx controller3
magicSeqIp.startEngine()

[cmd] start the engine
[cmd] start the successfully


In [20]:
print(magicSeqIp.printDebug())

----- MAIN STATUS ------------------
--------> STATUS =  SHUTDOWN
--------> MAINCNT =  0
--------> ENDCNT  =  1
--------> DMAADDR  =  0xa0000000
--------> DFXADDR  =  0xa0010000
----- SLOT DATA ------------------
------> slot 0 :
        srcAddr   : 0x187b000,  srcSize   : 0x40
        desAddr   : 0x187c000,  desSize   : 0x40
        status    : 0x0
        profileCnt: 0x1
------> slot 1 :
        srcAddr   : 0x187d000,  srcSize   : 0x40
        desAddr   : 0x187e000,  desSize   : 0x40
        status    : 0x0
        profileCnt: 0x1
------> slot 2 :
        srcAddr   : 0x0,  srcSize   : 0x0
        desAddr   : 0x0,  desSize   : 0x0
        status    : 0x0
        profileCnt: 0x0
------> slot 3 :
        srcAddr   : 0x0,  srcSize   : 0x0
        desAddr   : 0x0,  desSize   : 0x0
        status    : 0x0
        profileCnt: 0x0
-------------------------------
None
